## Ciscenje .csv filea

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', None)
df = pd.read_csv('vehicles.csv')

In [2]:
df_manu = pd.read_csv('VIN_manufacturer.csv')
df_manu.manufacturer = df_manu.manufacturer.apply(lambda x: x.split(',')[0])
for name in df['manufacturer'].unique():
    df_manu.manufacturer.loc[
        df_manu['manufacturer'].str.contains('|'.join(str(name).split('-')), case=False, na=False)]=name
    
manufacturer_ref = df_manu.groupby('manufacturer').id

### Koraci u analizi
https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-data?resource=download<br>Dropat cu county jer ionako ima samo null vrijednosti.<br>
Dropat cu sve stupce koji sadrze urlove, model nece moc izvuc neke bitne informacije iz toga a bio bi sporiji zbog dodatnih dimenzija.<br>
Dropat cu id.<br>
Dropat cu description, ipak ne radim nlp model, pre dug je string da bi model nasao sta korisno.<br>
Dropat cu size, pre malo redova ima podatak o tome da bi bilo korisno.<br><br>

Zamijeniti np.inf sa nullom.<br><br>

Pobrisat cu redove sa price==0, null i resetirat indekse.<br><br>

Datum cu pretvoriti u pandas datetime format i kasnije u format koji model moze procesirati.<br>
Izvuc broj cilindara iz stringa i pretvorit type u int.<br>

Vin broj cu rastaviti na komponente.<br><br>

Mislim da cu napraviti vise dataseta:
1. za skup podataka ociscen od svih redova s null
2. za skup podataka koji ce imat samo stupce s preko 400000 non-null vrijednosti
3. za skup podataka u kojem cu pobrisat sve redove s preko 3 null vrijednosti i zamijenit kasnije sve nullove sa median, 'unknown' ili slicno

In [3]:
df = df.drop(['county','description', 'size', 'id'], axis=1)
df = df.loc[:,~df.columns.str.contains('url', case=False)]

df = df.replace([np.inf, -np.inf], np.nan)


df.cylinders = df.cylinders.str.extract('(\d+)').astype('Int16')
df.year = df.year.astype('Int16')
df.odometer = df.odometer.astype('Int32')
df.price = df.price.astype('int32')

df = df[df.price != 0]
df = df.dropna(axis=0, subset=['price'])
df = df.reset_index(drop=True)

In [4]:
df_3 = df.dropna(thresh=df.shape[1] - 3)

In [9]:
diction = {}
for i in df_3.columns:
    diction[i] = df_3.loc[52][i]
    
diction

{'region': 'auburn',
 'price': 25590,
 'year': 2015,
 'manufacturer': 'gmc',
 'model': 'sierra 1500 regular cab',
 'condition': 'good',
 'cylinders': 6,
 'fuel': 'other',
 'odometer': 35290,
 'title_status': 'clean',
 'transmission': 'other',
 'VIN': '1GTN1TEH9FZ243281',
 'drive': nan,
 'type': 'pickup',
 'paint_color': 'white',
 'state': 'al',
 'lat': 32.59,
 'long': -85.48,
 'posting_date': '2021-04-28T11:01:06-0500'}

In [ ]:
df['posting_date'] = pd.to_datetime(df['posting_date'].str[0:-5])
df['posting_date']= df['posting_date'].map(dt.datetime.toordinal)

In [5]:
df['VIN_country'] = df.VIN.str[0:1]
df['VIN_manufacturer'] = df.VIN.str[1:3]
df['VIN_model_engine_type'] = df.VIN.str[3:8]
df['VIN_security_code'] = df.VIN.str[8:9]
df['VIN_year'] = df.VIN.str[9:10]
df['VIN_plant'] = df.VIN.str[10:11]
df = df.drop(['VIN'], axis=1)
df.replace('', np.nan, inplace=True)

In [6]:
df_3['VIN_country'] = df_3.VIN.str[0:1]
df_3['VIN_manufacturer'] = df_3.VIN.str[1:3]
df_3['VIN_model_engine_type'] = df_3.VIN.str[3:8]
df_3['VIN_security_code'] = df_3.VIN.str[8:9]
df_3['VIN_year'] = df_3.VIN.str[9:10]
df_3['VIN_plant'] = df_3.VIN.str[10:11]
df_3 = df_3.drop(['VIN'], axis=1)
df_3.replace('', np.nan, inplace=True)

In [7]:
df_3['VIN_temp'] = df_3.manufacturer.apply(
    lambda x: np.random.choice(
        list(set(manufacturer_ref.get_group(x)))) if x in manufacturer_ref.groups.keys() else float('NaN'))
df_3.VIN_country.fillna(df_3.VIN_temp.str[0:1], inplace = True)
df_3.VIN_manufacturer.fillna(df_3.VIN_temp.str[1:], inplace = True)
df_3 = df_3.drop(['VIN_temp'], axis=1)

In [8]:
df_1 = df.dropna()
df_2 = df.loc[:, (df.count() >= 350000)]
df_2 = df_2.dropna()

In [9]:
dtypes_1 = df_1.dtypes.apply(lambda x: x.name).to_dict()
dtypes_2 = df_2.dtypes.apply(lambda x: x.name).to_dict()
dtypes_3 = df_3.dtypes.apply(lambda x: x.name).to_dict()
dtypes_1

{'region': 'object',
 'price': 'int32',
 'year': 'Int16',
 'manufacturer': 'object',
 'model': 'object',
 'condition': 'object',
 'cylinders': 'Int16',
 'fuel': 'object',
 'odometer': 'Int32',
 'title_status': 'object',
 'transmission': 'object',
 'drive': 'object',
 'type': 'object',
 'paint_color': 'object',
 'state': 'object',
 'lat': 'float64',
 'long': 'float64',
 'posting_date': 'int64',
 'VIN_country': 'object',
 'VIN_manufacturer': 'object',
 'VIN_model_engine_type': 'object',
 'VIN_security_code': 'object',
 'VIN_year': 'object',
 'VIN_plant': 'object'}

In [10]:
df_1.to_csv('vehicles_1.csv', index=False)
df_2.to_csv('vehicles_2.csv', index=False)
df_3.to_csv('vehicles_3.csv', index=False)